In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data

In [5]:
DATA_TRAIN_PATH = 'data/train.csv'
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

## Load the testing data

In [6]:
DATA_TEST_PATH = 'data/test.csv' 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [7]:
X_train.shape, X_test.shape

((250000, 30), (568238, 30))

## Preprocessing and feature engineering

In [13]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3, encodable_th=0.7, switch_encoding=True)

In [9]:
tX_train.shape, tX_test.shape

((250000, 24), (568238, 24))

## Model selection and hyperparameter tuning

In [28]:
linear_param_grid = {
    'lambda_': 0,
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]
}
metrics_ls, params_ls = grid_search_cv(y_train, tX_train, model_fn=least_squares, loss_fn=compute_mse, predict_fn=predict_linear,
                          param_grid=linear_param_grid, transform_fn=build_poly, k_fold=5, seed=1)

LinAlgError: Singular matrix

In [29]:
metrics_ls_gd, params_ls_gd = grid_search_cv(y_train, tX_train, model_fn=least_squares_GD, loss_fn=compute_mse, predict_fn=predict_linear,
                          param_grid=linear_param_grid, transform_fn=build_poly, k_fold=5, seed=1)

/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:66: RuntimeWarning: overflow encountered in matmul
  return tx.T @ ((tx @ w) - y)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:66: RuntimeWarning: invalid value encountered in matmul
  return tx.T @ ((tx @ w) - y)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/helpers.py:117: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/helpers.py:118: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp / (tp + fn)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:94: RuntimeWarning: invalid value encountered in subtract
  w -= gamma * gradient


In [30]:
metrics_ls_gd

{'loss': nan, 'accuracy': 0.0, 'f1_score': nan}

In [31]:
metrics_ls_SGD, params_ls_SGD = grid_search_cv(y_train, tX_train, model_fn=least_squares_SGD, loss_fn=compute_mse, predict_fn=predict_linear,
                          param_grid=linear_param_grid, transform_fn=build_poly, k_fold=5, seed=1)

/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:66: RuntimeWarning: overflow encountered in matmul
  return tx.T @ ((tx @ w) - y)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:66: RuntimeWarning: invalid value encountered in matmul
  return tx.T @ ((tx @ w) - y)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/implementations.py:94: RuntimeWarning: invalid value encountered in subtract
  w -= gamma * gradient
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/helpers.py:117: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
/home/lucastrg/FLEP/MA1/ML/cs-433/ml-project-1-gradient_surfers/helpers.py:118: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp / (tp + fn)


In [32]:
metrics_ls_SGD

{'loss': nan, 'accuracy': 0.0, 'f1_score': nan}

In [33]:
ridge_param_grid = {
    'lambda_': np.logspace(-4, 0, 5),
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]
}
metrics_ridge, params_ridge = ridge_regression_cv(ty_train, tX_train, param_grid=ridge_param_grid)


In [34]:
metrics_ridge

{'loss': 0.08810935635844389,
 'accuracy': 33.974000000000004,
 'f1_score': 0.5281876571663158}

In [21]:
log_param_grid = {
    'lambda_': np.logspace(-4, 0, 5),
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]
}
metrics_rlg, params_rlg = logistic_regression_cv(ty_train, tX_train, param_grid=log_param_grid)


In [22]:
metrics_rlg

{'loss': 0.48832368060485065,
 'accuracy': 76.5648,
 'f1_score': 0.6261136027521852}

In [35]:
metrics_lg, params_lg = logistic_regression_cv(ty_train, tX_train, {'lambda_':0, 'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]})

In [36]:
metrics_lg

{'loss': 0.496098732815953,
 'accuracy': 76.1192,
 'f1_score': 0.6243271235036281}

## Run selected model

In [15]:
tX_train_poly_rlg = build_poly(tX_train, degree=params_rlg['degree'], cont_features=cont_features)
weights_rlg, loss_rlg = reg_logistic_regression(ty_train, tX_train_poly_rlg, max_iters=1000, lambda_=params_rlg['lambda_'])

In [16]:
ty_train_pred = predict_logistic(weights_rlg, tX_train_poly_rlg)
train_accuracy = compute_accuracy(ty_train, ty_train_pred)
train_f1 = compute_f1(ty_train, ty_train_pred)
train_accuracy, train_f1

(78.54560000000001, 0.6573436401967674)

## Prepare test data for prediction

In [17]:
tX_test_poly = build_poly(tX_test, params['degree'], cont_features)

In [ ]:
tX_test_poly.shape

(568238, 44)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
from datetime import datetime
method = 'reg_logistic_regression'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'submissions/submission_{method}_{time}'
y_pred = predict_logistic(weights, tX_test_poly)
y_pred = replace_values(y_pred, from_val=0, to_val=-1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)